In [1]:
import akshare as ak
import pandas as pd
import os
import baostock as bs

In [2]:
bs.login()

login success!


In [3]:
fund_etf_category_sina_df = ak.fund_etf_category_sina(symbol="ETF基金")

In [ ]:
fund_etf_category_sina_df.to_csv('data/fund_etf_category_sina.csv')

In [5]:
etf_sina_savepath = 'etf_sina'
if not os.path.exists(etf_sina_savepath):
    os.mkdir(etf_sina_savepath)

In [6]:
etf_codes = fund_etf_category_sina_df['代码'].to_list()

In [54]:

# for fund_code in etf_codes:
#     try:
#         df = ak.fund_etf_hist_sina(fund_code)
#         df['pctChg'] = df['close'].pct_change().fillna(0)
#         df['code'] = fund_code
#         df = df[['date', 'code', 'open', 'high', 'low', 'close', 'volume', 'pctChg']]
#         df.to_csv(f'{etf_sina_savepath}/{fund_code}.csv', index=False)
#     except:
#         print(fund_code)
#         continue

In [ ]:
def get_fund_etf_hist_sina(code):
    try:
        return pd.read_csv(f"{etf_sina_savepath}/{code}.csv")
    except:
        try:
            df = ak.fund_etf_hist_sina(code)
            df['pctChg'] = df['close'].pct_change().fillna(0)
            df['code'] = code
            df = df[['date', 'code', 'open', 'high', 'low', 'close', 'volume', 'pctChg']]
            df.to_csv(f'{etf_sina_savepath}/{code}.csv', index=False)
            return df
        except:
            print(f"{code} not found")

In [ ]:
fund_etf_category_sina_returns = {}
for code in etf_codes:
    try:
        fund_etf_category_sina_returns[code] = get_fund_etf_hist_sina(code)['pctChg']
    except:
        print(code)
        continue

sz159399
sz159393
sz159201
sh589990
sh589900
sh589890
sh589880
sh589860
sh589800
sh589770
sh589680
sh589660
sh589600
sh589500
sh589080
sh589000
sh588960
sh588840
sh588240
sh520580
sh513830


In [61]:
returns_df = pd.DataFrame(fund_etf_category_sina_returns).dropna(axis=1, how='all')

In [62]:
corr_matrix = returns_df.corr(method='pearson')

In [66]:
threshold = 0.8  # 相似度阈值
to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > threshold:
            # 保留规模较大的基金
            fund_i = corr_matrix.columns[i]
            fund_j = corr_matrix.columns[j]
            if fund_etf_category_sina_df.loc[fund_etf_category_sina_df['代码'] == fund_i, '成交额'].values[0] \
                > fund_etf_category_sina_df.loc[fund_etf_category_sina_df['代码'] == fund_j, '成交额'].values[0]:
                to_drop.add(fund_j)
            else:
                to_drop.add(fund_i)


In [67]:
len(to_drop)

437

In [70]:
filtered_codes = [code for code in returns_df.columns if code not in to_drop]

In [71]:
len(filtered_codes)

639

In [ ]:
final_df = fund_etf_category_sina_df[fund_etf_category_sina_df['代码'].isin(filtered_codes)]
final_df = final_df.sort_values(by='成交额', ascending=False).head(300)  # 取前300名


            代码        名称          成交额
979   sh511990   华宝添益ETF  11872072987
886   sh513330  恒生互联网ETF   9920216755
1017  sh511090  30年国债ETF   9494815243
988   sh511880   银华日利ETF   8736808019
1006  sh511360     短融ETF   8064568159
...        ...       ...          ...
851   sh513970   恒生消费ETF     45487028
211   sz159707     地产ETF     45147727
1003  sh511580  国债政金债ETF     44207094
596   sh561160    锂电池ETF     44031113
466   sh588380   双创50ETF     43885184

[300 rows x 3 columns]


In [ ]:
final_df[['代码', '名称']].to_csv('data/fund_etf_top300_sina.csv', index=False)